In [6]:
import os
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage
from langchain.callbacks.tracers import LangChainTracer
from langchain_anthropic import ChatAnthropic
from langchain import PromptTemplate

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
#model = ChatAnthropic(model="claude-3-haiku-20240307", anthropic_api_key=anthropic_api_key)
model = ChatAnthropic(model="claude-3-5-sonnet-20240620", anthropic_api_key=anthropic_api_key, max_tokens=4096)
# Set up LangChain tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "daily"

In [44]:
# employee_profiles.py

employee_profiles = {
    "Sarah Chen": {
        "role": "Project Manager / Scrum Master",
        "responsibilities": [
            "Oversee project progress and timelines",
            "Facilitate communication between team members and stakeholders",
            "Conduct sprint planning, daily stand-ups, and retrospectives",
            "Manage project risks and resources"
        ],
        "skills": [
            "Certified Scrum Master",
            "Proficient in Agile methodologies",
            "Experienced with project management tools (Jira, Trello)",
            "Basic understanding of both frontend (React) and backend (Node.js) development",
            "Excellent communication and leadership skills"
        ]
    },
    "Alex Rodriguez": {
        "role": "Senior Full-Stack Developer",
        "responsibilities": [
            "Lead technical decisions and architecture design",
            "Mentor junior developers",
            "Implement complex features across the stack",
            "Code review and quality assurance"
        ],
        "skills": [
            "Expert in React.js, Node.js, and Express.js",
            "Proficient in database design and management (PostgreSQL)",
            "Experienced with cloud services (AWS)",
            "Strong problem-solving and system design skills",
            "Knowledgeable in security best practices and performance optimization"
        ]
    },
    "Emily Watson": {
        "role": "Frontend Developer",
        "responsibilities": [
            "Implement responsive and intuitive user interfaces",
            "Collaborate with UI/UX designer to bring designs to life",
            "Optimize frontend performance",
            "Ensure cross-browser compatibility"
        ],
        "skills": [
            "Expert in React.js and TypeScript",
            "Proficient in HTML5, CSS3, and JavaScript",
            "Experienced with state management (Redux, MobX)",
            "Knowledgeable in frontend testing frameworks (Jest, React Testing Library)",
            "Familiar with UI component libraries and CSS-in-JS solutions"
        ]
    },
    "Michael Kim": {
        "role": "Backend Developer",
        "responsibilities": [
            "Develop and maintain server-side logic",
            "Design and implement database schemas",
            "Create and document APIs",
            "Implement data processing and analysis features"
        ],
        "skills": [
            "Expert in Node.js and Express.js",
            "Proficient in database management (PostgreSQL, MongoDB)",
            "Experienced with RESTful API design and GraphQL",
            "Knowledgeable in data structures and algorithms",
            "Familiar with microservices architecture"
        ]
    },
    "Olivia Martinez": {
        "role": "QA Engineer / DevOps Specialist",
        "responsibilities": [
            "Develop and execute test plans and test cases",
            "Set up and maintain CI/CD pipelines",
            "Manage deployment processes and infrastructure",
            "Monitor system performance and security"
        ],
        "skills": [
            "Proficient in manual and automated testing (Selenium, Cypress)",
            "Experienced with containerization (Docker) and orchestration (Kubernetes)",
            "Knowledgeable in cloud platforms (AWS, Google Cloud)",
            "Familiar with monitoring tools (ELK stack, Prometheus)",
            "Basic scripting skills (Python, Bash)"
        ]
    },
    "Liam Foster": {
        "role": "UI/UX Designer",
        "responsibilities": [
            "Create user-centric designs and prototypes",
            "Conduct user research and usability testing",
            "Develop and maintain the design system",
            "Collaborate with developers to ensure design integrity"
        ],
        "skills": [
            "Expert in design tools (Figma, Adobe Creative Suite)",
            "Proficient in interaction design and prototyping",
            "Experienced with user research methodologies",
            "Knowledgeable in accessibility standards",
            "Basic understanding of HTML and CSS"
        ]
    }
}

import json

def load_employee_profiles(file_path):
    """
    Load employee profiles from a JSON file.
    
    Args:
    file_path (str): Path to the JSON file containing employee profiles.
    
    Returns:
    dict: A dictionary of employee profiles.
    """
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file {file_path}")
        return {}

# Example usage:
# state["employee_profiles"] = load_employee_profiles('path/to/employee_profiles.json')
# 
# for employee, profile in state["employee_profiles"].items():
#     formatted_prompt = PERSONAL_TASK_EXTRACTOR_PROMPT.format(
#         note=state["note"],
#         employee_profile=profile
#     )
#     # Use formatted_prompt as needed

In [47]:
emp = load_employee_profiles(file_path="../data/company-data/employee-profiles.json")


<class 'dict'>


{'role': 'Project Manager / Scrum Master',
 'responsibilities': ['Oversee project progress and timelines',
  'Facilitate communication between team members and stakeholders',
  'Conduct sprint planning, daily stand-ups, and retrospectives',
  'Manage project risks and resources'],
 'skills': ['Certified Scrum Master',
  'Proficient in Agile methodologies',
  'Experienced with project management tools (Jira, Trello)',
  'Basic understanding of both frontend (React) and backend (Node.js) development',
  'Excellent communication and leadership skills']}

In [41]:

from typing import Dict

class DailyState(TypedDict):
    project_sprint_state: str
    project_backlog: str
    employee_profiles: Dict
    note: str
    general_info : str
    reports: Dict[str, str]  # Key: employee name, Value: report content
    diagrams: Dict[str, str]  # Key: employee name, Value: Mermaid diagram code
    personal_tasks : Dict [str,str]

In [78]:
import os
import base64
import requests

def load_str_from_txt(file_path: str) -> str:
    with open(file_path, 'r') as file:
        return file.read()
    
def load_markdown_to_str(file_path):
    with open(file_path, 'r', encoding='utf-8') as md_file:
        markdown_content = md_file.read()
    return markdown_content

def render_mermaid_diagram(diagram_code: str) -> str:
    # Encode the Mermaid code
    encoded_diagram = base64.b64encode(diagram_code.encode('utf-8')).decode('utf-8')
    
    # Make a request to the Mermaid rendering service
    url = f"https://mermaid.ink/img/{encoded_diagram}"
    response = requests.get(url)
    
    if response.status_code == 200:
        # Return the URL of the rendered image
        return url
    else:
        # If rendering failed, return the original Mermaid code
        return f"```mermaid\n{diagram_code}\n```"
    
def format_mermaid(input_string):
    # Step 1: Remove redundant "```mermaid" at the start and end
    cleaned_string = input_string.replace('```mermaid', '').replace('```', '')
    
    # Step 2: Replace escaped newlines with actual newlines
    formatted_string = cleaned_string.replace(r'\n', '\n')
    
    # Step 3: Strip any leading/trailing whitespace
    formatted_string = formatted_string.strip()

    return formatted_string	

In [79]:
render_mermaid_diagram(diagram_code=format_mermaid(state["diagrams"]["Alex Rodriguez"]))

'https://mermaid.ink/img/Z3JhcGggVEQKICAgIHN1YmdyYXBoIENvbXBsZXRlZAogICAgZW5kCgogICAgc3ViZ3JhcGggSW5fUHJvZ3Jlc3MKICAgICAgICBCW0RhdGEgTW9kZWwgU2NhbGFiaWxpdHkgQ29uY2VybnNdCiAgICAgICAgQ1tUaGlyZC1wYXJ0eSBBUEkgSW50ZWdyYXRpb25dCiAgICBlbmQKCiAgICBzdWJncmFwaCBVcGNvbWluZwogICAgICAgIERbUmVzb2x2ZSBDSS9DRCBQaXBlbGluZSBJc3N1ZXNdCiAgICAgICAgRVtQcm90b3R5cGUgU2NhbGFiaWxpdHkgU29sdXRpb25zXQogICAgICAgIEZbRm9sbG93IHVwIG9uIEFQSSBEb2N1bWVudGF0aW9uXQogICAgICAgIEdbUHJlcGFyZSBmb3IgU3ByaW50IFBsYW5uaW5nXQogICAgZW5kCgogICAgQiAtLT58TGVhZHMgdG98IEUKICAgIEMgLS0+fExlYWRzIHRvfCBGCiAgICBEIC0tPiBHCiAgICBFIC0tPiBHCiAgICBGIC0tPiBH'

In [35]:
GENERAL_INFO_EXTRACTOR_PROMPT = """
You are an AI assistant helping to extract general information from a daily scrum meeting note. Your task is to summarize the key points that are relevant to all team members.


Here's the daily scrum meeting note:
####
{note}
####

Please extract and summarize the following information:
1. Overall sprint progress
2. Main challenges or blockers
3. Key decisions made
4. Any updates that affect the entire team

Provide this information in a concise, bullet-point format.
IMPORTANT: Answer only with the general information. No more. 
You must not say here is the summary ... just the information.
Note that your task is not to make a summary but to find information that effect everyone.
"""

In [36]:
PERSONAL_TASK_EXTRACTOR_PROMPT = """
You are an AI assistant helping to extract personal tasks and progress for a specific team member from a daily scrum meeting note.

Here's the daily scrum meeting note:
####
{note}
####

Here's the profile of the team member:
####
{employee_profile}
####

Here's the curret state of the project: 
Note that this document already contains information about the current meeting that have just happened. It is up to date.
####
{project_sprint_state}
####

Here's the current state of the backlog:
Note that this document does not contains information about the current meeting. It representes a previous state.
####
{project_backlog}
####

Please extract the following information for this team member:
1. Their reported progress since the last meeting
2. Their current tasks or focus areas
3. Any blockers or challenges they're facing
4. Their next steps or goals

Provide this information in a concise, bullet-point format.
"""

In [61]:

DIAGRAM_GENERATOR_PROMPT = """
You are an AI assistant tasked with generating a Mermaid diagram based on a team member's tasks and progress.

Here's the daily scrum meeting note:
####
{note}
####


Here's the information about the team member's tasks and progress:
####
{personal_tasks}
####

Please create a Mermaid diagram that visualizes this information. The diagram should show:
1. Completed tasks
2. In-progress tasks
3. Upcoming tasks
4. Any dependencies between tasks

Provide only the Mermaid diagram code, without any explanation or additional text.
"""


In [80]:

REPORT_COMPILER_PROMPT = """
You are an AI assistant tasked with compiling a personalized daily scrum report for a team member.

Here's the general project information:
####
{general_info}
####

Here's the team member's personal task information:
####
{personal_tasks}
####

Here's a rendered Mermaid diagram representing the team member's tasks:
![Task Diagram]({diagram})

Please compile a personalized report for this team member. The report should include:
1. A brief overview of the sprint progress
2. The team member's personal progress and next steps
3. Any relevant challenges or blockers
4. The rendered Mermaid diagram (include it using the Markdown image syntax)

Format the report in Markdown, including appropriate headers and sections.
"""


In [63]:

MOTIVATIONAL_MESSAGE_PROMPT = """
You are an AI assistant tasked with generating a short, personalized motivational message or fun fact for a team member.

Here's the team member's profile:
####
{employee_profile}
####

Please generate a short, uplifting message or interesting fun fact related to the team member's interests, role, or recent achievements. The message should be positive and encouraging.

Provide only the message, without any additional explanation.
"""

In [15]:
note = """
Thank you for the request to review and improve the meeting notes. After careful consideration, I believe some adjustments are necessary to ensure accuracy, completeness, and clarity. Here's a revised version of the notes:\n\nMeeting Notes: Sprint 1 Review and Retrospective - HealthTrack Pro\n\nDate: July 3, 2023\nTime: 2:00 PM - 4:00 PM\nAttendees: Sarah Chen (PM/Scrum Master), Alex Rodriguez (Senior Full-Stack Developer), Emily Watson (Frontend Developer), Michael Kim (Backend Developer), Olivia Martinez (QA/DevOps Specialist), Liam Foster (UI/UX Designer)\n\n1. Sprint 1 Overview\n   - Sprint Goal: Implement core user authentication and health dashboard functionality for HealthTrack Pro's MVP\n   - Sprint Duration: June 20, 2023 to July 3, 2023 (10 working days)\n   - Committed Capacity: 99 Story Points\n   - Completed: 56 Story Points\n\n2. Sprint 1 Accomplishments\n\n   2.1 User Authentication System (41 Story Points)\n   \n   Backend (Michael Kim):\n   - Completed user model and authentication logic\n   - Implemented API endpoints for registration and login\n   - Integrated JWT for session management\n   \n   Frontend (Emily Watson):\n   - Implemented registration and login forms with validation\n   - Integrated authentication state management using React Context API\n   - Implemented protected routes for authenticated users\n\n   Security (Olivia Martinez):\n   - Implemented password hashing with bcrypt\n   - Set up rate limiting for login attempts\n   - Started work on automated security scans in CI/CD pipeline\n\n   Design (Liam Foster):\n   - Created login and registration interface designs\n   - Implemented responsive design for desktop and mobile\n\n   2.2 Health Dashboard (15 Story Points)\n   \n   Frontend (Emily Watson):\n   - Set up basic dashboard structure using React components\n   - Implemented responsive grid layout\n   - Integrated Chart.js for data visualization\n   - Implemented basic charts for step counts and sleep data\n\n   Backend (Michael Kim):\n   - Set up API endpoints to serve dashboard data\n   - Implemented data aggregation functions for daily, weekly, and monthly metrics\n\n   Design (Liam Foster):\n   - Finalized overall layout and color scheme (60% complete)\n   - Designed custom icons for health metrics and activities\n   - Created sections for daily activity summary, nutrition overview, sleep patterns, and goal progress\n\n3. Incomplete Items\n   - Basic Activity Tracking (30 Story Points): Not started\n   - Password reset functionality for User Authentication\n   - Two-factor authentication implementation\n   - Detailed views for each health category in Dashboard design\n   - Backend integration for activity data visualization\n\n4. Challenges and Impediments\n   - CI/CD Pipeline Issues (Olivia Martinez):\n     - Intermittent failures in end-to-end tests\n     - Docker containerization issues in replicating production environment\n     - Compatibility issues with security scanning tools integration\n   - Data Model Scalability Concerns (Alex Rodriguez):\n     - Potential performance issues with large datasets\n     - Need for data archiving strategy\n   - Third-party API Integration (Alex Rodriguez):\n     - Delays in receiving updated documentation from API provider\n\n5. Sprint Retrospective\n\n   5.1 What Went Well\n   - Strong collaboration between frontend and design teams\n   - Effective daily stand-ups for alignment and blocker resolution\n   - Good progress on development infrastructure setup\n\n   5.2 Areas for Improvement\n   - More conservative estimations for complex features or new technologies\n   - Enhanced sprint planning process with thorough task breakdown\n   - Allocate more time for knowledge sharing among team members\n\n6. Action Items for Next Sprint\n   - Olivia and Alex: Collaborate on resolving CI/CD pipeline issues\n   - Michael and Alex: Prototype scalability solutions for data model\n   - Emily: Research and propose formal estimation process for sprint planning\n   - Liam: Schedule mid-sprint design review\n   - Alex: Follow up on third-party API documentation\n   - All: Prepare for sprint planning session (July 4, 2023, at 10 AM)\n\n7. Next Meeting: Sprint Planning Session - July 4, 2023, at 10:00 AM\n\n
"""

In [48]:
# Initialize state
state = DailyState(
    project_sprint_state=load_markdown_to_str(file_path="../data/company-data/current-project-sprint-state.md"),
    project_backlog=load_markdown_to_str(file_path="../data/company-data/current-project-backlog.md"),
    employee_profiles=load_employee_profiles(file_path="../data/company-data/employee-profiles.json"),
    note=note,
    general_info= "",
    reports={},
    diagrams={},
    personal_tasks = {}
)
state

{'project_sprint_state': '# Project Sprint State Template\n\n## Project Name: HealthTrack Pro\n\n### Sprint Information\n- Sprint Number: 1\n- Sprint Goal: "Implement core user authentication and health dashboard functionality for HealthTrack Pro\'s MVP"\n- Sprint Duration: June 20, 2023 to July 3, 2023\n- Total Working Days: 10\n\n### Team Capacity\n- Team Members: 6\n- Total Capacity (Story Points): 99\n- Committed Capacity (Story Points): 99\n\n### Sprint Backlog\n| User Story | Story Points | Status | Assigned To | Notes |\n|------------|--------------|--------|-------------|-------|\n| User Authentication - Backend | 13 | Done | Michael Kim | User model and authentication logic completed, API endpoints implemented |\n| User Authentication - Frontend | 10 | Done | Emily Watson | Registration and login forms with validation completed, authentication state management implemented using React Context API |\n| User Authentication - Security | 5 | In Progress | Olivia Martinez | Password

In [49]:
def general_info_extractor_node(state: DailyState) -> DailyState:
    formatted_prompt = GENERAL_INFO_EXTRACTOR_PROMPT.format(
        note=state["note"]
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Extract the general information from the daily scrum note.")
    ]
    response = model.invoke(messages)
    state["general_info"] = response.content
    return state


In [50]:
state = general_info_extractor_node(state=state)
state

{'project_sprint_state': '# Project Sprint State Template\n\n## Project Name: HealthTrack Pro\n\n### Sprint Information\n- Sprint Number: 1\n- Sprint Goal: "Implement core user authentication and health dashboard functionality for HealthTrack Pro\'s MVP"\n- Sprint Duration: June 20, 2023 to July 3, 2023\n- Total Working Days: 10\n\n### Team Capacity\n- Team Members: 6\n- Total Capacity (Story Points): 99\n- Committed Capacity (Story Points): 99\n\n### Sprint Backlog\n| User Story | Story Points | Status | Assigned To | Notes |\n|------------|--------------|--------|-------------|-------|\n| User Authentication - Backend | 13 | Done | Michael Kim | User model and authentication logic completed, API endpoints implemented |\n| User Authentication - Frontend | 10 | Done | Emily Watson | Registration and login forms with validation completed, authentication state management implemented using React Context API |\n| User Authentication - Security | 5 | In Progress | Olivia Martinez | Password

In [53]:

def personal_task_extractor_node(state: DailyState) -> DailyState:
    for employee, profile in state["employee_profiles"].items():
        formatted_prompt = PERSONAL_TASK_EXTRACTOR_PROMPT.format(
            note=state["note"],
            employee_profile=profile,
            project_sprint_state = state.get("project_sprint_state"),
            project_backlog = state.get("project_backlog")


        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content=f"Extract personal tasks for {employee}.")
        ]
        response = model.invoke(messages)
        state["personal_tasks"][employee] = response.content
    return state


In [54]:
state = personal_task_extractor_node(state=state)

In [56]:
state["personal_tasks"]["Alex Rodriguez"]

"Based on the provided information, here's the extracted personal tasks for Alex Rodriguez, the Senior Full-Stack Developer:\n\n1. Reported progress since last meeting:\n- No specific progress reported for Alex in the meeting notes\n\n2. Current tasks or focus areas:\n- Collaborating on data model scalability concerns\n- Addressing potential performance issues with large datasets\n\n3. Blockers or challenges:\n- Data Model Scalability Concerns:\n  * Potential performance issues with large datasets\n  * Need for data archiving strategy\n- Third-party API Integration:\n  * Delays in receiving updated documentation from API provider\n\n4. Next steps or goals:\n- Collaborate with Olivia on resolving CI/CD pipeline issues\n- Work with Michael to prototype scalability solutions for data model\n- Follow up on third-party API documentation\n- Prepare for sprint planning session (July 4, 2023, at 10 AM)"

In [65]:
def diagram_generator_node(state: DailyState) -> DailyState:
    for employee, tasks in state["personal_tasks"].items():
        formatted_prompt = DIAGRAM_GENERATOR_PROMPT.format(
            note = state.get("note"),
			personal_tasks=tasks
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content=f"Generate a Mermaid diagram for {employee}'s tasks.")
        ]
        response = model.invoke(messages)
        state["diagrams"][employee] = response.content
    return state

In [59]:
before_diagram_generator_node = state

In [66]:
state = diagram_generator_node(state=state)
state

{'project_sprint_state': '# Project Sprint State Template\n\n## Project Name: HealthTrack Pro\n\n### Sprint Information\n- Sprint Number: 1\n- Sprint Goal: "Implement core user authentication and health dashboard functionality for HealthTrack Pro\'s MVP"\n- Sprint Duration: June 20, 2023 to July 3, 2023\n- Total Working Days: 10\n\n### Team Capacity\n- Team Members: 6\n- Total Capacity (Story Points): 99\n- Committed Capacity (Story Points): 99\n\n### Sprint Backlog\n| User Story | Story Points | Status | Assigned To | Notes |\n|------------|--------------|--------|-------------|-------|\n| User Authentication - Backend | 13 | Done | Michael Kim | User model and authentication logic completed, API endpoints implemented |\n| User Authentication - Frontend | 10 | Done | Emily Watson | Registration and login forms with validation completed, authentication state management implemented using React Context API |\n| User Authentication - Security | 5 | In Progress | Olivia Martinez | Password

In [68]:
state["diagrams"]["Alex Rodriguez"]

'```mermaid\ngraph TD\n    subgraph Completed\n    end\n\n    subgraph In_Progress\n        B[Data Model Scalability Concerns]\n        C[Third-party API Integration]\n    end\n\n    subgraph Upcoming\n        D[Resolve CI/CD Pipeline Issues]\n        E[Prototype Scalability Solutions]\n        F[Follow up on API Documentation]\n        G[Prepare for Sprint Planning]\n    end\n\n    B -->|Leads to| E\n    C -->|Leads to| F\n    D --> G\n    E --> G\n    F --> G\n```'

In [89]:

def report_compiler_node(state: DailyState) -> DailyState:
    for employee in state["employee_profiles"].keys():
        rendered_diagram = render_mermaid_diagram(format_mermaid(state["diagrams"][employee]))
        print(format_mermaid(state["diagrams"][employee]))
        print("ÁÁÁÁ", rendered_diagram)

        formatted_prompt = REPORT_COMPILER_PROMPT.format(
            general_info=state["general_info"],
            personal_tasks=state["personal_tasks"][employee],
            diagram=rendered_diagram
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content=f"Compile a personalized report for {employee}.")
        ]
        response = model.invoke(messages)
        state["reports"][employee] = response.content
    return state

In [90]:
state = report_compiler_node(state=state)
state["reports"]

graph TD
    subgraph Completed
        A[Facilitate Sprint 1 Review and Retrospective]
        B[Oversee completion of 56/99 Story Points]
    end
    
    subgraph In Progress
        C[Manage project risks and resources]
        D[Facilitate team communication]
        E[Oversee project progress and timelines]
    end
    
    subgraph Upcoming
        F[Prepare for Sprint Planning Session]
        G[Implement conservative estimations]
        H[Enhance sprint planning process]
        I[Allocate time for knowledge sharing]
        J[Ensure follow-up on action items]
    end
    
    A --> C
    B --> C
    C --> F
    D --> F
    E --> F
    F --> G
    F --> H
    F --> I
    F --> J
ÁÁÁÁ https://mermaid.ink/img/Z3JhcGggVEQKICAgIHN1YmdyYXBoIENvbXBsZXRlZAogICAgICAgIEFbRmFjaWxpdGF0ZSBTcHJpbnQgMSBSZXZpZXcgYW5kIFJldHJvc3BlY3RpdmVdCiAgICAgICAgQltPdmVyc2VlIGNvbXBsZXRpb24gb2YgNTYvOTkgU3RvcnkgUG9pbnRzXQogICAgZW5kCiAgICAKICAgIHN1YmdyYXBoIEluIFByb2dyZXNzCiAgICAgICAgQ1tNYW5hZ2UgcHJvamVjdCBya

KeyboardInterrupt: 

In [91]:
state["reports"]["Alex Rodriguez"]

"# Daily Scrum Report for Alex Rodriguez\n\n## Sprint Overview\nWe're currently in Sprint 1, which has seen progress with 56 out of 99 committed Story Points completed. The team is facing some challenges, particularly with CI/CD pipeline issues, data model scalability concerns, and delays in receiving third-party API documentation.\n\n## Personal Progress and Next Steps\n\n### Completed Tasks\nNo specific completed tasks were reported for Alex in the last meeting.\n\n### In Progress\n1. **Data Model Scalability Concerns**\n   - Addressing potential performance issues with large datasets\n   - Collaborating on developing scalability solutions\n\n2. **Third-party API Integration**\n   - Awaiting updated documentation from the API provider\n\n### Upcoming Tasks\n1. Collaborate with Olivia on resolving CI/CD pipeline issues\n2. Work with Michael to prototype scalability solutions for the data model\n3. Follow up on third-party API documentation\n4. Prepare for the sprint planning session o

In [73]:
def motivational_message_node(state: DailyState) -> DailyState:
    for employee, profile in state["employee_profiles"].items():
        formatted_prompt = MOTIVATIONAL_MESSAGE_PROMPT.format(
            employee_profile=profile
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content=f"Generate a motivational message for {employee}.")
        ]
        response = model.invoke(messages)
        state["reports"][employee] += f"\n\n{response.content}"
    return state

In [74]:
state = motivational_message_node(state=state)
state["reports"]["Liam Foster"]

"# Daily Scrum Report for Liam Foster (UI/UX Designer)\n\n## Sprint Overview\nWe're currently in Sprint 1, which has completed 56 out of 99 committed Story Points. The team is facing challenges with CI/CD pipeline issues, data model scalability, and delays in receiving third-party API documentation. Our next Sprint Planning Session is scheduled for July 4, 2023, at 10:00 AM.\n\n## Personal Progress\n\n### Completed Tasks\n- Designed login and registration interfaces with responsive design\n- Created overall layout and color scheme for health dashboard (60% complete)\n- Designed custom icons for health metrics and activities\n- Developed sections for daily activity summary, nutrition overview, sleep patterns, and goal progress\n\n### Current Focus\n- Finalizing the health dashboard design\n- Working on detailed views for each health category in Dashboard design\n\n### Next Steps\n- Complete the remaining 40% of the overall layout and color scheme for the health dashboard\n- Start workin

In [75]:
def create_daily_scrum_report_workflow(checkpointer):
    workflow = StateGraph(DailyState)
    
    # Define nodes
    workflow.add_node("general_info_extractor", general_info_extractor_node)
    workflow.add_node("personal_task_extractor", personal_task_extractor_node)
    workflow.add_node("diagram_generator", diagram_generator_node)
    workflow.add_node("report_compiler", report_compiler_node)
    workflow.add_node("motivational_message", motivational_message_node)
    
    # Define edges
    workflow.add_edge("general_info_extractor", "personal_task_extractor")
    workflow.add_edge("personal_task_extractor", "diagram_generator")
    workflow.add_edge("diagram_generator", "report_compiler")
    workflow.add_edge("report_compiler", "motivational_message")
    workflow.add_edge("motivational_message", END)
    
    # Set entry point
    workflow.set_entry_point("general_info_extractor")
    
    return workflow.compile(checkpointer=checkpointer)

In [92]:
import os
from datetime import datetime

def run_daily_step():
    # Initialize state
    state = DailyState(
        project_sprint_state=load_markdown_to_str(file_path="../data/company-data/current-project-sprint-state.md"),
        project_backlog=load_markdown_to_str(file_path="../data/company-data/current-project-backlog.md"),
        employee_profiles=load_employee_profiles(file_path="../data/company-data/employee-profiles.json"),
        note=note,
        general_info= "",
        reports={},
        diagrams={},
        personal_tasks = {}
    )
    all_states = []
    with SqliteSaver.from_conn_string(":memory:") as memory:
        thread = {"configurable": {"thread_id": "1"}}
        graph = create_daily_scrum_report_workflow(memory)
        for s in graph.stream(state, thread):
            print(s)
            all_states.append(s)
        last_state = {key: value for key, value in all_states[-1].items()}["motivational_message"]
    # Create the daily-reports directory if it doesn't exist
    os.makedirs("daily-reports", exist_ok=True)

    # Get the current date for the filename
    current_date = datetime.now().strftime("%Y-%m-%d")

    # Log and export reports
    for employee, report in last_state["reports"].items():
        print(f"Report for {employee}:")
        print(report)
        print("\n---\n")

        # Export to file in the daily-reports directory (note the removal of the leading slash)
        filename = f"daily-reports/{current_date}_{employee}_daily_report.md"
        with open(filename, "w") as f:
            f.write(report)
        print(f"Report saved to {filename}")

In [84]:
state["diagrams"]["Alex Rodriguez"]

'```mermaid\ngraph TD\n    subgraph Completed\n    end\n\n    subgraph In_Progress\n        B[Data Model Scalability Concerns]\n        C[Third-party API Integration]\n    end\n\n    subgraph Upcoming\n        D[Resolve CI/CD Pipeline Issues]\n        E[Prototype Scalability Solutions]\n        F[Follow up on API Documentation]\n        G[Prepare for Sprint Planning]\n    end\n\n    B -->|Leads to| E\n    C -->|Leads to| F\n    D --> G\n    E --> G\n    F --> G\n```'

In [87]:
render_mermaid_diagram(format_mermaid(state["diagrams"]["Sarah Chen"]))

'https://mermaid.ink/img/Z3JhcGggVEQKICAgIHN1YmdyYXBoIENvbXBsZXRlZAogICAgICAgIEFbRmFjaWxpdGF0ZSBTcHJpbnQgMSBSZXZpZXcgYW5kIFJldHJvc3BlY3RpdmVdCiAgICAgICAgQltPdmVyc2VlIGNvbXBsZXRpb24gb2YgNTYvOTkgU3RvcnkgUG9pbnRzXQogICAgZW5kCiAgICAKICAgIHN1YmdyYXBoIEluIFByb2dyZXNzCiAgICAgICAgQ1tNYW5hZ2UgcHJvamVjdCByaXNrcyBhbmQgcmVzb3VyY2VzXQogICAgICAgIERbRmFjaWxpdGF0ZSB0ZWFtIGNvbW11bmljYXRpb25dCiAgICAgICAgRVtPdmVyc2VlIHByb2plY3QgcHJvZ3Jlc3MgYW5kIHRpbWVsaW5lc10KICAgIGVuZAogICAgCiAgICBzdWJncmFwaCBVcGNvbWluZwogICAgICAgIEZbUHJlcGFyZSBmb3IgU3ByaW50IFBsYW5uaW5nIFNlc3Npb25dCiAgICAgICAgR1tJbXBsZW1lbnQgY29uc2VydmF0aXZlIGVzdGltYXRpb25zXQogICAgICAgIEhbRW5oYW5jZSBzcHJpbnQgcGxhbm5pbmcgcHJvY2Vzc10KICAgICAgICBJW0FsbG9jYXRlIHRpbWUgZm9yIGtub3dsZWRnZSBzaGFyaW5nXQogICAgICAgIEpbRW5zdXJlIGZvbGxvdy11cCBvbiBhY3Rpb24gaXRlbXNdCiAgICBlbmQKICAgIAogICAgQSAtLT4gQwogICAgQiAtLT4gQwogICAgQyAtLT4gRgogICAgRCAtLT4gRgogICAgRSAtLT4gRgogICAgRiAtLT4gRwogICAgRiAtLT4gSAogICAgRiAtLT4gSQogICAgRiAtLT4gSg=='

In [93]:
run_daily_step()

{'general_info_extractor': {'project_sprint_state': '# Project Sprint State Template\n\n## Project Name: HealthTrack Pro\n\n### Sprint Information\n- Sprint Number: 1\n- Sprint Goal: "Implement core user authentication and health dashboard functionality for HealthTrack Pro\'s MVP"\n- Sprint Duration: June 20, 2023 to July 3, 2023\n- Total Working Days: 10\n\n### Team Capacity\n- Team Members: 6\n- Total Capacity (Story Points): 99\n- Committed Capacity (Story Points): 99\n\n### Sprint Backlog\n| User Story | Story Points | Status | Assigned To | Notes |\n|------------|--------------|--------|-------------|-------|\n| User Authentication - Backend | 13 | Done | Michael Kim | User model and authentication logic completed, API endpoints implemented |\n| User Authentication - Frontend | 10 | Done | Emily Watson | Registration and login forms with validation completed, authentication state management implemented using React Context API |\n| User Authentication - Security | 5 | In Progress |